In [224]:
%reset -f
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [225]:
import numpy as np
from scipy.sparse import lil_matrix


def generate_full_rank_sparse_integer_system(N, density=0.01, value_range=(-10, 10), random_state=None):
    """
    フルランクなスパース整数連立一次方程式 Ax = b を生成する。
    
    N: 変数（=行数=列数）
    density: 全体に占める非ゼロ要素割合
    value_range: 非ゼロ整数値の範囲（例：(-5, 5)）
    """
    if random_state is not None:
        np.random.seed(random_state)

    A = lil_matrix((N, N), dtype=int)

    # ステップ1: 各行に1つ以上の非ゼロ（対角線に1を置くことでrankを確保）
    for i in range(N):
        val = 0
        while val == 0:
            val = np.random.randint(value_range[0], value_range[1] + 1)
        A[i, i] = val

    # ステップ2: ランダムな位置に追加で非ゼロ整数を入れる（スパース性の調整）
    total_nonzeros_target = int(N * N * density)
    current_nonzeros = N  # すでに対角にN個ある
    while current_nonzeros < total_nonzeros_target:
        i = np.random.randint(0, N)
        j = np.random.randint(0, N)
        if A[i, j] == 0:
            val = 0
            while val == 0:
                val = np.random.randint(value_range[0], value_range[1] + 1)
            A[i, j] = val
            current_nonzeros += 1

    # bベクトルも整数で
    b = np.random.randint(value_range[0], value_range[1] + 1, size=N)
    if np.linalg.matrix_rank(A.toarray()) < N:
        return generate_full_rank_sparse_integer_system(N, density, value_range, random_state)

    return A.tocsr(), b

N = 5
A, b = generate_full_rank_sparse_integer_system(N, density=0.5, value_range=(-5, 5))
print(f"A.shape = {A.shape}, non zero = {A.nnz}, rank = {np.linalg.matrix_rank(A.toarray())}")
print(A.toarray()) 
print(b)

A.shape = (5, 5), non zero = 12, rank = 5
[[-3  0  0 -3  0]
 [ 0  4  0  0 -3]
 [ 0  0  4  0  0]
 [ 0  0  1  2  0]
 [-5 -2 -3  4  1]]
[-1  0  5  4  4]


In [226]:
from mechanics import *
import string

vars = string.ascii_lowercase[:N]

system = (
    System()
    .add_variable(' '.join(vars))
)
for i in range(N):
    system.equate(
        '+'.join([f'{A[i, j]}*{vars[j]}' for j in range(N)]),
        b[i],
    )
system.show_all()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [227]:
solver = system.solver()
# solver.plot_dependencies()
result = solver.run({})

Indices: ()
match [Eq_0 <- a, {}, Eq_1 <- e, {}, Eq_2 <- c, {}, Eq_3 <- d, {}, Eq_4 <- b, {}]
Edges: ['Eq_0 -> d, a', 'a -> Eq_0', 'Eq_1 -> e, b', 'e -> Eq_1', 'Eq_2 -> c', 'c -> Eq_2', 'Eq_3 -> d, c', 'd -> Eq_3', 'Eq_4 -> d, a, c, b, e', 'b -> Eq_4']
defaultdict(<class 'dict'>, {Eq_2: 4*c = 5: {c: 4}})
defaultdict(<class 'dict'>, {Eq_3: c + 2*d = 4: {d: 2}})
defaultdict(<class 'dict'>, {Eq_0: -3*a - 3*d = -1: {a: -3}})
defaultdict(<class 'dict'>, {Eq_4: -5*a - 2*b - 3*c + 4*d + e = 4: {e: 1, b: -2}, Eq_1: 4*b - 3*e = 0: {e: -3, b: 4}})
Block #0(equations=('Eq_2',), unknowns=(c,)), knowns=())
Block #1(equations=('Eq_3',), unknowns=(d,)), knowns=(c,))
Block #2(equations=('Eq_0',), unknowns=(a,)), knowns=(d,))
Block #3(equations=('Eq_4', 'Eq_1'), unknowns=(e, b)), knowns=(d, a, c))

module constants
    real(8), parameter :: pi = 3.14159265358979323846
end module constants

module variables
    real(8), save :: a = 0.0
    real(8), save :: b = 0.0
    real(8), save :: c = 0.0
    real(8

ld: warning: ignoring duplicate libraries: '-lgfortran'


 Started
 Output in /Users/yuuki.fj/Develop/mechanics/examples/result/20250716_031740/
  Block 0 converged in   1 iterations, residual = 0.0000E+00
  Block 1 converged in   1 iterations, residual = 0.0000E+00
  Block 2 converged in   1 iterations, residual = 0.4441E-15
  Block 3 converged in   1 iterations, residual = 0.3553E-14
 Completed


In [ ]:
x = np.array([ result[var] for var in vars ])
print(f"x = {x}")
print(f"Ax = {A.dot(x)}")
print(f"b = {b}")
np.testing.assert_allclose(A.dot(x), b, atol=1e-10)

x = [-1.04166667  4.4375      1.25        1.375       5.91666667]
Ax = [-1.00000000e+00  3.55271368e-15  5.00000000e+00  4.00000000e+00
  4.00000000e+00]
b = [-1  0  5  4  4]


AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 1 / 5 (20%)
Max absolute difference: 3.55271368e-15
Max relative difference: 4.4408921e-16
 x: array([-1.000000e+00,  3.552714e-15,  5.000000e+00,  4.000000e+00,
        4.000000e+00])
 y: array([-1,  0,  5,  4,  4])